<a href="https://colab.research.google.com/github/gibsonx/jlpt_simulator/blob/dev/graphs/n3/outliner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
if 'google.colab' in str(get_ipython()):
    !git clone https://github.com/gibsonx/jlpt_simulator.git
    %cd jlpt_simulator
    !git checkout dev
    
    !apt-get install python3-dev graphviz libgraphviz-dev pkg-config
    !pip install -r requirements.txt
else:
  print('Not running on CoLab')


Not running on CoLab


In [17]:
import json
import random
import datetime
import os
import logging
import pickle
import re
import uuid
import threading
import asyncio
from typing import *
from tqdm import tqdm
import time
import yaml
import sys
import asyncio
import json
import random
import time
from tqdm.asyncio import tqdm_asyncio
from graphs.common.utils import collect_vocabulary
from langchain_openai import AzureOpenAI,AzureChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from langchain.embeddings.base import Embeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_community.embeddings import XinferenceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from typing import Annotated, Literal, Sequence
from typing_extensions import TypedDict
from libs.LLMs import *
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from typing import Annotated, Sequence
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage,RemoveMessage,HumanMessage,AIMessage,ToolMessage
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List, Optional

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

from graphs.n3.ExamTaskHandler import ExamTaskHandler
load_dotenv()

True

In [18]:
# Import N3 Vocabulary
file_path = 'Vocab/n3.csv'
# Display the content of the CSV file
vocab_dict = collect_vocabulary(file_path)
with open("Vocab/topics.txt", "r", encoding="utf-8") as file:
    topics_list = [line.strip() for line in file]

vocab_dict

'{"優れる":"すぐれる","臨む":"のぞむ","美人":"びじん","ドライブ":"ドライブ","設計":"せっけい","今日":"こんにち","上":"かみ","ワイン":"ワイン","大半":"たいはん","捲る":"めくる","埋める":"うめる","違反":"いはん","同時":"どうじ","出場":"しゅつじょう","いつのまにか":"いつのまにか","マーケット":"マーケット","引用":"いんよう","刺す":"さす","穏やか":"おだやか","下":"しも","局":"きょく","情報":"じょうほう","後輩":"こうはい","工場":"こうば","節":"ふし","歌謡":"かよう","効果":"こうか","奇妙":"きみょう","ビデオ":"ビデオ","東洋":"とうよう","公平":"こうへい","トラック":"トラック","裂ける":"さける","労働":"ろうどう","タイプライター":"タイプライター","以前":"いぜん","航空":"こうくう","訓練":"くんれん","サイン":"サイン","清む":"すむ","熱中":"ねっちゅう","存在":"そんざい","付き合い":"つきあい","健康":"けんこう","まぶしい":"まぶしい","パイプ":"パイプ","そのうち":"そのうち","予期":"よき","破産":"はさん","優秀":"ゆうしゅう","精算":"せいさん","新興":"しんこう","協調":"きょうちょう","茶":"ちゃ","こぼす":"こぼす","梅":"うめ","緊張":"きんちょう","罰する":"ばっする","けち":"けち","札":"さつ","広まる":"ひろまる","生年月日":"せいねんがっぴ","日光":"にっこう","ヨーロッパ":"ヨーロッパ","塀":"へい","決まり":"きまり","作曲":"さっきょく","土":"つち","創造":"そうぞう","操作":"そうさ","こんなに":"こんなに","比較":"ひかく","流れ":"ながれ","外交":"がいこう","骨":"ほね","ユーモア":"ユーモア","表現":"ひょうげん","溺れる":"おぼれる","ほっと":"ほっと","期待":"きたい","異常":"いじょう","否定":"ひてい","消費":"しょうひ

#### load Models

In [19]:
from typing import List, Optional

from langchain_core.prompts import ChatPromptTemplate

from pydantic import BaseModel, Field

# instruction = """
# Section 1: Vocabulary and Grammar
# - Kanji reading (kanji_reading): 8 questions
# - Write Chinese characters (write_chinese): 6 questions
# - Word Meaning Selection (word_meaning): 11 questions
# - Synonyms substitution (synonym_substitution): 5 questions
# - word usage (word_usage): 5 questions
# - Grammar fill in the blank (sentence_grammar): 13 questions
# - Sentence sorting (sentence_sort): 5 questions
# - Grammar structure selection (sentence_structure): 4-5 questions
#
# Section 2: Reading Comprehension
# - Short passages (short_passage_read): 4 articles
# - Mid-size passages (midsize_passage_read): 2 articles
# - Long passages (long_passage_read): 1 articles
# - Information retrieval (info_retrieval): 1 articles
#
# Section 3: Listening Comprehension
# - Topic understanding (topic_understanding): 6 questions
# - Key understanding (keypoint_understanding): 6 questions
# - Summary understanding (summary_understanding): 3 questions
# - Active expression (active_expression): 4 questions
# - Immediate acknowledgment (immediate_ack): 9 questions
# """

instruction = """
Section 1: Vocabulary and Grammar
- Kanji reading (kanji_reading): 1 questions
- Write Chinese characters (write_chinese): 1 questions

Section 2: Reading Comprehension
- Information retrieval (info_retrieval): 1 articles

Section 3: Listening Comprehension
- Topic understanding (topic_understanding): 1 questions

"""


direct_gen_outline_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            f"You are a japanese teacher. Your job is to write an outline for a JLPT Japanese-Language Proficiency Test N3 level exam paper in Japanese. the complexity should be restricted to N3 level and respect japanese culture. The JLPT Japanese-Language Proficiency Test exam paper includes a mix of easy, moderate, and difficult questions to accurately assess the test-taker's proficiency across different aspects of the language."
            f"First,  randomly pick Japanese words in 'Vocabulary' for questions in Vocabulary and Grammar. At Section 2 and 3, randomly choose topics in 'TopicList' for Reading Comprehension and Listening Comprehension Sections, don't repeat to choose a same word or topic"
            f"Second, you should abide by the provided exam instruction and decide the number of questions and content in the each Section."
            f"Finally, write the outline of the examination paper in japanese and provide question topics according to the instructions."
            f"Instruction: {instruction}",
        ),
        ("user", "TopicList: {topic_list}, Vocabulary: {vocab_dict}"),
    ]
)


## Data Strcuture

In [20]:
class QuestionTopic(BaseModel):
    topic: str = Field(..., title="a vocabulary or topic hint for a question")


class Subsection(BaseModel):
    subsection_title: str = Field(..., title="subsection English word in () only from the instruction")
    description: str = Field(..., title="giving the number of questions and requirements")
    question_topics: Optional[List[QuestionTopic]] = Field(
        default_factory=list
    )

    @property
    def as_str(self) -> str:
        question_topics_str = "\n".join(
            f"- **{qt.topic}**" for qt in self.question_topics
        )
        return f"### {self.subsection_title}\n\n{self.description}\n\n{question_topics_str}".strip()

class Section(BaseModel):
    section_title: str = Field(..., title="Title of the section")
    subsections: Optional[List[Subsection]] = Field(
        default_factory=list,
        title="Titles and reason for each subsection of the JLPT exam page.",
    )

    @property
    def as_str(self) -> str:
        subsections = "\n\n".join(
            subsection.as_str for subsection in self.subsections or []
        )
        return f"## {self.section_title}\n\n{subsections}".strip()


class Outline(BaseModel):
    page_title: str = Field(..., title="Title of the JLPT exam page")
    sections: List[Section] = Field(
        default_factory=list,
        title="Titles and descriptions for each section of the JLPT exam paper.",
    )

    @property
    def as_str(self) -> str:
        sections = "\n\n".join(section.as_str for section in self.sections)
        return f"# {self.page_title}\n\n{sections}".strip()


In [21]:
# Read the topics from a file, sort them, and print the sorted list
def process_topics(file_path):
    try:
        # Read the file
        with open(file_path, 'r', encoding='utf-8') as file:
            topics = file.readlines()

        # Remove any extra whitespace or newline characters
        topics = [topic.strip() for topic in topics if topic.strip()]

        # Shuffle the topics randomly
        random.shuffle(topics)

    except FileNotFoundError:
        print("The file was not found. Please check the file path.")
    except Exception as e:
        print("An error occurred:", str(e))

    except FileNotFoundError:
        print("The file was not found. Please check the file path.")
    except Exception as e:
        print("An error occurred:", str(e))

In [22]:
# Preload all topics from the file
with open("Vocab/topics.txt", "r", encoding="utf-8") as file:
    topics_list = [line.strip() for line in file]

generate_outline_direct = direct_gen_outline_prompt | azure_llm.with_structured_output(Outline)
initial_outline = generate_outline_direct.invoke({"topic_list": topics_list, "vocab_dict": vocab_dict})

2025-05-23 13:35:13,007 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


In [23]:
from IPython.display import display, Markdown, Latex
display(Markdown(initial_outline.as_str))

# 日本語能力試験N3 模擬試験 問題概要

## 第1部：語彙・文法

### 漢字読み (kanji_reading)

1問。与えられた漢字の正しい読み方を選びなさい。

- **優れる**
- **埋める**

### 漢字書き (write_chinese)

1問。ひらがなで書かれた言葉の正しい漢字を書きなさい。

- **違反**

## 第2部：読解

### 情報検索 (info_retrieval)

1つの短い文章を読んで、内容に関する情報を正しく選びなさい。

- **レストランで食べ物を注文する**

## 第3部：聴解

### 話題理解 (topic_understanding)

1問。会話を聞いて、話題が何かを選びなさい。

- **道を尋ねる**

In [24]:
outliner_json = initial_outline.model_dump_json()
data = json.loads(outliner_json)  # Replace with your actual JSON data

start_time = time.time()

output_data = {'sections': []}

for section in data['sections']:
    output_section = {'section_title': section['section_title'], 'subsections': []}

    for subsection in tqdm(section['subsections'], desc=f"Processing {section['section_title']}"):
        function_name = subsection['subsection_title'].split(' (')[1].rstrip(')')
        questions = subsection['question_topics']

        for question in tqdm(questions, desc=f"Processing {subsection['subsection_title']}"):
            handler = ExamTaskHandler()
            func = getattr(handler, function_name, None)

            if func:
                max_attempts = 2
                original_topic = question['topic']  # Optional: Track original topic
                for attempt in range(max_attempts):
                    try:
                        result = func(question['topic'])
                        question['result'] = result
                        break  # Exit on success
                    except Exception as e:
                        if attempt < max_attempts - 1:
                            # Replace topic and retry
                            question['topic'] = random.choice(topics_list)
                        else:
                            question['result'] = f"Error after {max_attempts} attempts: {str(e)}"
            else:
                question['result'] = f"Method {function_name} not found"

        output_subsection = {
            'subsection_title': subsection['subsection_title'],
            'description': subsection['description'],
            'question_topics': questions
        }
        output_section['subsections'].append(output_subsection)

    output_data['sections'].append(output_section)

# End the timer
end_time = time.time()

# Calculate the total execution time
execution_time = end_time - start_time

print(f"Total execution time: {execution_time:.2f} seconds")

Processing 第1部：語彙・文法:   0%|          | 0/2 [00:00<?, ?it/s]

---WEB SEARCH---
---Generator----


2025-05-23 13:35:20,500 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


---REVISOR---


2025-05-23 13:35:27,470 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


...

---REVISOR---


2025-05-23 13:37:53,542 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


--- AI Reviser feels Good Enough ---
--- Formatter ---


2025-05-23 13:37:56,863 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
Processing 第3部：聴解: 100%|██████████| 1/1 [00:20<00:00, 20.48s/it]

Total execution time: 163.79 seconds


In [25]:
yaml_str = yaml.dump(output_data, allow_unicode=True, sort_keys=False, default_flow_style=False)
# Display as formatted YAML
display(Markdown(yaml_str))

sections:
- section_title: 第1部：語彙・文法
  subsections:
  - subsection_title: 漢字読み (kanji_reading)
    description: 1問。与えられた漢字の正しい読み方を選びなさい。
    question_topics:
    - topic: 優れる
      result:
        question: 次の下線部の漢字の読み方として最もよいものを一つ選びなさい。彼は語学に 優れる 人です。
        choices:
        - すぐれる
        - まさる
        - いだれる
        - ゆうれる
        correct_answer: 1
        explanation: 「優れる」は「すぐれる」と読みます。意味は「他よりも能力や価値、技量などが高いこと」です。選択肢の「まさる」は「勝る」と書き、意味は似ていますが漢字が異なります。「いだれる」「ゆうれる」は実際には存在しない読みです。学習者が間違えやすいポイントは、「優（ゆう）」の音読みに引っ張られてしまうことや、「勝る（まさる）」との混同です。文脈と漢字の正しい読みをしっかり覚えましょう。
    - topic: 埋める
      result:
        question: 穴を土で **埋める**。
        choices:
        - うめる
        - うずめる
        - おさめる
        - いれる
        correct_answer: 1
        explanation: 「埋める」は「うめる」と読みます。似た読み方に「うずめる」があり、どちらも物を土や他のもので覆い隠す意味がありますが、「穴を土で埋める」のような物理的な行為では、一般的に「うめる」が使われます。「うずめる」も物理的な意味で使われることがありますが、比喩的な表現（例：「顔を枕にうずめる」）や決まった言い回しで使われることが多いです。選択肢3「おさめる」は「収める」「納める」などの漢字で使われ、「しまう」「おさめる」などの意味です。選択肢4「いれる」は「入れる」で、「中に入れる」意味です。どちらも「埋める」の読みや意味とは異なりますが、N3レベルの学習者が誤用しやすい語を選択肢に入れています。N3レベルでは、同じ漢字に複数の読みがある場合や、似た意味の語の使い分けに注意が必要です。例えば、「埋める（うめる）」と「埋める（うずめる）」のように、文脈によって正しい読みや使い方が変わる語がよく出題されます。具体例：「穴を土で埋める」（うめる：物理的に土を入れて穴をふさぐ）、「顔を枕に埋める」（うずめる：顔を隠すように押し付ける）。このように、文脈に応じた読み方の選択が重要です。
  - subsection_title: 漢字書き (write_chinese)
    description: 1問。ひらがなで書かれた言葉の正しい漢字を書きなさい。
    question_topics:
    - topic: 違反
      result:
        question: この道路でスピードを出しすぎると、交通の＿＿になります。
        choices:
        - 違反
        - 違法
        - 違約
        - 違和
        correct_answer: 1
        explanation: この問題は、「いはん」という語の正しい漢字表記を選ぶものです。「違反」は、法律や規則、約束などにそむくことを意味し、「交通の違反」という表現が最も自然です。選択肢の「違法」は「法律に反すること」ですが、「交通の違法」とは通常言いません。「違約」は約束を破ること、「違和」は気分や雰囲気が合わないことを指し、いずれも文脈に合いません。似た漢字語の意味と使い方の違いに注意する必要があり、N3レベルの学習者にとって良い練習となります。
- section_title: 第2部：読解
  subsections:
  - subsection_title: 情報検索 (info_retrieval)
    description: 1つの短い文章を読んで、内容に関する情報を正しく選びなさい。
    question_topics:
    - topic: レストランで食べ物を注文する
      result:
        html_article: '### レストラン「さくら」新メニューと利用案内 | メニュー名 | 値段 | 提供時間 | 特徴・注意事項 | |--------------------|-----------|----------------|--------------------------------------|
          | 日替わりランチ | 900円 | 11:00-14:00 | 毎日内容が変わります。数量限定です。 | | ハンバーグ定食 | 1,200円
          | 終日 | サラダとご飯付き | | 魚の煮付け定食 | 1,100円 | 11:00-20:00 | 小鉢と味噌汁付き | | お子さまランチ
          | 700円 | 11:00-18:00 | 小学生以下のお子さま限定 | | デザートセット | 600円 | 14:00-17:00 | ケーキと飲み物のセット
          | **ご案内と注意事項：** - ご注文はテーブルで「すみません」と店員を呼んでからお願いします。 - 日替わりランチは数量限定のため、売り切れの場合があります。
          - お子さまランチは小学生以下のお子さまのみご注文いただけます。 - デザートセットは午後2時から午後5時までの提供です。 - アレルギーがある方は、注文時に店員にお知らせください。
          --- #### 問題 A. 日替わりランチを食べたい場合、どの時間にレストランに行くのがよいですか。 1. 午前9時 2. 午前11時 3.
          午後3時 4. 午後8時 B. ケーキと飲み物のセットを注文したいとき、どの時間に行くのが正しいですか。 1. 午前10時 2. 午後1時 3.
          午後3時 4. 午後8時 C. アレルギーがあるお客様は、どのようにしたらよいですか。 1. メニューを見ない 2. 店員に伝える 3. 何も言わない
          4. 他のお客さんに聞く D. 店員を呼びたいとき、どの表現が正しいですか。 1. 「こんにちは」と言う 2. 「すみません」と言う 3. 「ありがとう」と言う
          4. 「ごちそうさま」と言う E. 次の会話の（　）に入る言葉として正しいものを選びなさい。 お客さん：「（　　　）、日替わりランチをください。」
          店員：「かしこまりました。」 1. ありがとう 2. すみません 3. いらっしゃいませ 4. こんにちは --- #### 解答・解説 Aの正解：2（午前11時）
          **解説**：日替わりランチは「11:00-14:00」まで提供です。午前9時や午後3時、午後8時は提供時間外です。N3では時刻表現の理解がポイントです。
          Bの正解：3（午後3時） **解説**：デザートセットは「14:00-17:00」に提供されます。午後3時はこの時間内です。午前や午後8時は時間外、午後1時もまだ提供前です。
          Cの正解：2（店員に伝える） **解説**：「アレルギーがある方は、注文時に店員にお知らせください」と注意書きがあります。日本語で「伝える」「お知らせください」が読めるかがポイントです。
          Dの正解：2（「すみません」と言う） **解説**：「ご注文はテーブルで『すみません』と店員を呼んでからお願いします」と書かれています。「すみません」はレストランで店員を呼ぶときの定番表現です。
          Eの正解：2（すみません） **解説**：注文のときは「すみません、○○をください」と言うのが丁寧です。「ありがとう」は注文前には使いません。
          --- #### 教師からの解説 今回の問題は、レストランの利用案内やメニュー表から必要な情報を探し、正しく理解・応用できるかを問う内容です。N3レベルの学習者にとって、時刻表現・注意書き・会話表現の読解が主なチャレンジポイントとなります。「すみません」の使い方や、アレルギー対応の案内も日本の実際のレストラン文化に即しており、実用的な日本語力を測る良い練習となります。'
        questions:
        - question: 日替わりランチを食べたい場合、どの時間にレストランに行くのがよいですか。
          choices:
          - 午前9時
          - 午前11時
          - 午後3時
          - 午後8時
          correct_answer: 2
          explanation: 日替わりランチは「11:00-14:00」まで提供です。午前9時や午後3時、午後8時は提供時間外です。N3では時刻表現の理解がポイントです。
        - question: ケーキと飲み物のセットを注文したいとき、どの時間に行くのが正しいですか。
          choices:
          - 午前10時
          - 午後1時
          - 午後3時
          - 午後8時
          correct_answer: 3
          explanation: デザートセットは「14:00-17:00」に提供されます。午後3時はこの時間内です。午前や午後8時は時間外、午後1時もまだ提供前です。
        - question: アレルギーがあるお客様は、どのようにしたらよいですか。
          choices:
          - メニューを見ない
          - 店員に伝える
          - 何も言わない
          - 他のお客さんに聞く
          correct_answer: 2
          explanation: 「アレルギーがある方は、注文時に店員にお知らせください」と注意書きがあります。日本語で「伝える」「お知らせください」が読めるかがポイントです。
        - question: 店員を呼びたいとき、どの表現が正しいですか。
          choices:
          - 「こんにちは」と言う
          - 「すみません」と言う
          - 「ありがとう」と言う
          - 「ごちそうさま」と言う
          correct_answer: 2
          explanation: 「ご注文はテーブルで『すみません』と店員を呼んでからお願いします」と書かれています。「すみません」はレストランで店員を呼ぶときの定番表現です。
        - question: 次の会話の（　）に入る言葉として正しいものを選びなさい。お客さん：「（　　　）、日替わりランチをください。」店員：「かしこまりました。」
          choices:
          - ありがとう
          - すみません
          - いらっしゃいませ
          - こんにちは
          correct_answer: 2
          explanation: 注文のときは「すみません、○○をください」と言うのが丁寧です。「ありがとう」は注文前には使いません。
- section_title: 第3部：聴解
  subsections:
  - subsection_title: 話題理解 (topic_understanding)
    description: 1問。会話を聞いて、話題が何かを選びなさい。
    question_topics:
    - topic: 道を尋ねる
      result:
        conversation:
        - gender: male
          context: 男の人と女の人が道で話しています。男の人は美術館に行きたいようです。
        - gender: male
          context: すみません、美術館へはどうやって行けばいいですか。
        - gender: female
          context: この道をまっすぐ行って、二つ目の信号を右に曲がってください。
        - gender: male
          context: 二つ目の信号を右ですね。
        - gender: female
          context: はい。右に曲がると、すぐ左側に大きな公園があります。その公園の中を通って行くと、美術館の入り口がありますよ。
        - gender: male
          context: ありがとうございます。歩いてどのくらいかかりますか。
        - gender: female
          context: そうですね、10分ぐらいだと思います。
        - gender: male
          context: わかりました。助かりました。
        questions:
          question: この会話の後、男の人はまず何をしますか。
          choices:
          - 右に曲がって公園に入る
          - バスに乗る
          - この道をまっすぐ歩く
          - 近くの人にもう一度道を聞く
          correct_answer: 3
          explanation: この問題のポイントは、道案内の基本的な表現（「まっすぐ行って」「二つ目の信号を右に曲がる」など）を正しく聞き取れるかどうかです。選択肢の中には、すぐに右に曲がる（1）や公共交通機関を使う（2）、もう一度道を聞く（4）など、よくある混乱しやすいパターンが含まれています。正しい順序を聞き取り、最初の行動を選ぶことが求められます。N3レベルでは、道案内の会話を聞いて流れを理解できるかが重要なポイントです。


In [26]:
output_data

{'sections': [{'section_title': '第1部：語彙・文法',
   'subsections': [{'subsection_title': '漢字読み (kanji_reading)',
     'description': '1問。与えられた漢字の正しい読み方を選びなさい。',
     'question_topics': [{'topic': '優れる',
       'result': {'question': '次の下線部の漢字の読み方として最もよいものを一つ選びなさい。彼は語学に 優れる 人です。',
        'choices': ['すぐれる', 'まさる', 'いだれる', 'ゆうれる'],
        'correct_answer': 1,
        'explanation': '「優れる」は「すぐれる」と読みます。意味は「他よりも能力や価値、技量などが高いこと」です。選択肢の「まさる」は「勝る」と書き、意味は似ていますが漢字が異なります。「いだれる」「ゆうれる」は実際には存在しない読みです。学習者が間違えやすいポイントは、「優（ゆう）」の音読みに引っ張られてしまうことや、「勝る（まさる）」との混同です。文脈と漢字の正しい読みをしっかり覚えましょう。'}},
      {'topic': '埋める',
       'result': {'question': '穴を土で **埋める**。',
        'choices': ['うめる', 'うずめる', 'おさめる', 'いれる'],
        'correct_answer': 1,
        'explanation': '「埋める」は「うめる」と読みます。似た読み方に「うずめる」があり、どちらも物を土や他のもので覆い隠す意味がありますが、「穴を土で埋める」のような物理的な行為では、一般的に「うめる」が使われます。「うずめる」も物理的な意味で使われることがありますが、比喩的な表現（例：「顔を枕にうずめる」）や決まった言い回しで使われることが多いです。選択肢3「おさめる」は「収める」「納める」などの漢字で使われ、「しまう」「おさめる」などの意味です。選択肢4「いれる」は「入れる」で、「中に入れる」意味です。どちらも「埋

In [27]:
def render_to_html(sections):
    html = '<html><body>\n'
    for section in sections:
        html += f'<h1>{section.get("section_title", "")}</h1>\n'
        for subsection in section.get("subsections", []):
            html += f'<h2>{subsection.get("subsection_title", "")}</h2>\n'
            if "description" in subsection:
                html += f'<p>{subsection["description"]}</p>\n'
            for qtopic in subsection.get("question_topics", []):
                html += f'<h3>{qtopic.get("topic", "")}</h3>\n'
                result = qtopic.get("result", {})
                if isinstance(result, dict):
                    if "html_article" in result:
                        html += result["html_article"] + "\n"
                    if "question" in result:
                        html += f'<p><strong>Q: {result["question"]}</strong></p>\n'
                        if "choices" in result:
                            correct = result.get("correct_answer", -1)
                            html += '<ul>\n'
                            for idx, choice in enumerate(result["choices"], 1):
                                if idx == correct:
                                    html += f"<li><b>{choice}</b> <span style='color:green;'>(correct)</span></li>\n"
                                else:
                                    html += f"<li>{choice}</li>\n"
                            html += '</ul>\n'
                        if "explanation" in result:
                            html += f'<p><em>Explanation: {result["explanation"]}</em></p>\n'
                    if "questions" in result and isinstance(result["questions"], list):
                        for q in result["questions"]:
                            html += f'<p><strong>Q: {q["question"]}</strong></p>\n'
                            if "choices" in q:
                                correct = q.get("correct_answer", -1)
                                html += '<ul>\n'
                                for idx, choice in enumerate(q["choices"], 1):
                                    if idx == correct:
                                        html += f"<li><b>{choice}</b> <span style='color:green;'>(correct)</span></li>\n"
                                    else:
                                        html += f"<li>{choice}</li>\n"
                                html += '</ul>\n'
                            if "explanation" in q:
                                html += f'<p><em>Explanation: {q["explanation"]}</em></p>\n'
                    if "questions" in result and isinstance(result["questions"], dict):
                        q = result["questions"]
                        html += f'<p><strong>Q: {q["question"]}</strong></p>\n'
                        if "choices" in q:
                            correct = q.get("correct_answer", -1)
                            html += '<ul>\n'
                            for idx, choice in enumerate(q["choices"], 1):
                                if idx == correct:
                                    html += f"<li><b>{choice}</b> <span style='color:green;'>(correct)</span></li>\n"
                                else:
                                    html += f"<li>{choice}</li>\n"
                            html += '</ul>\n'
                        if "explanation" in q:
                            html += f'<p><em>Explanation: {q["explanation"]}</em></p>\n'
    html += '</body></html>'
    return html

In [28]:
html_output = render_to_html(output_data['sections'])
display(Markdown(html_output))

<html><body>
<h1>第1部：語彙・文法</h1>
<h2>漢字読み (kanji_reading)</h2>
<p>1問。与えられた漢字の正しい読み方を選びなさい。</p>
<h3>優れる</h3>
<p><strong>Q: 次の下線部の漢字の読み方として最もよいものを一つ選びなさい。彼は語学に 優れる 人です。</strong></p>
<ul>
<li><b>すぐれる</b> <span style='color:green;'>(correct)</span></li>
<li>まさる</li>
<li>いだれる</li>
<li>ゆうれる</li>
</ul>
<p><em>Explanation: 「優れる」は「すぐれる」と読みます。意味は「他よりも能力や価値、技量などが高いこと」です。選択肢の「まさる」は「勝る」と書き、意味は似ていますが漢字が異なります。「いだれる」「ゆうれる」は実際には存在しない読みです。学習者が間違えやすいポイントは、「優（ゆう）」の音読みに引っ張られてしまうことや、「勝る（まさる）」との混同です。文脈と漢字の正しい読みをしっかり覚えましょう。</em></p>
<h3>埋める</h3>
<p><strong>Q: 穴を土で **埋める**。</strong></p>
<ul>
<li><b>うめる</b> <span style='color:green;'>(correct)</span></li>
<li>うずめる</li>
<li>おさめる</li>
<li>いれる</li>
</ul>
<p><em>Explanation: 「埋める」は「うめる」と読みます。似た読み方に「うずめる」があり、どちらも物を土や他のもので覆い隠す意味がありますが、「穴を土で埋める」のような物理的な行為では、一般的に「うめる」が使われます。「うずめる」も物理的な意味で使われることがありますが、比喩的な表現（例：「顔を枕にうずめる」）や決まった言い回しで使われることが多いです。選択肢3「おさめる」は「収める」「納める」などの漢字で使われ、「しまう」「おさめる」などの意味です。選択肢4「いれる」は「入れる」で、「中に入れる」意味です。どちらも「埋める」の読みや意味とは異なりますが、N3レベルの学習者が誤用しやすい語を選択肢に入れています。N3レベルでは、同じ漢字に複数の読みがある場合や、似た意味の語の使い分けに注意が必要です。例えば、「埋める（うめる）」と「埋める（うずめる）」のように、文脈によって正しい読みや使い方が変わる語がよく出題されます。具体例：「穴を土で埋める」（うめる：物理的に土を入れて穴をふさぐ）、「顔を枕に埋める」（うずめる：顔を隠すように押し付ける）。このように、文脈に応じた読み方の選択が重要です。</em></p>
<h2>漢字書き (write_chinese)</h2>
<p>1問。ひらがなで書かれた言葉の正しい漢字を書きなさい。</p>
<h3>違反</h3>
<p><strong>Q: この道路でスピードを出しすぎると、交通の＿＿になります。</strong></p>
<ul>
<li><b>違反</b> <span style='color:green;'>(correct)</span></li>
<li>違法</li>
<li>違約</li>
<li>違和</li>
</ul>
<p><em>Explanation: この問題は、「いはん」という語の正しい漢字表記を選ぶものです。「違反」は、法律や規則、約束などにそむくことを意味し、「交通の違反」という表現が最も自然です。選択肢の「違法」は「法律に反すること」ですが、「交通の違法」とは通常言いません。「違約」は約束を破ること、「違和」は気分や雰囲気が合わないことを指し、いずれも文脈に合いません。似た漢字語の意味と使い方の違いに注意する必要があり、N3レベルの学習者にとって良い練習となります。</em></p>
<h1>第2部：読解</h1>
<h2>情報検索 (info_retrieval)</h2>
<p>1つの短い文章を読んで、内容に関する情報を正しく選びなさい。</p>
<h3>レストランで食べ物を注文する</h3>
### レストラン「さくら」新メニューと利用案内 | メニュー名 | 値段 | 提供時間 | 特徴・注意事項 | |--------------------|-----------|----------------|--------------------------------------| | 日替わりランチ | 900円 | 11:00-14:00 | 毎日内容が変わります。数量限定です。 | | ハンバーグ定食 | 1,200円 | 終日 | サラダとご飯付き | | 魚の煮付け定食 | 1,100円 | 11:00-20:00 | 小鉢と味噌汁付き | | お子さまランチ | 700円 | 11:00-18:00 | 小学生以下のお子さま限定 | | デザートセット | 600円 | 14:00-17:00 | ケーキと飲み物のセット | **ご案内と注意事項：** - ご注文はテーブルで「すみません」と店員を呼んでからお願いします。 - 日替わりランチは数量限定のため、売り切れの場合があります。 - お子さまランチは小学生以下のお子さまのみご注文いただけます。 - デザートセットは午後2時から午後5時までの提供です。 - アレルギーがある方は、注文時に店員にお知らせください。 --- #### 問題 A. 日替わりランチを食べたい場合、どの時間にレストランに行くのがよいですか。 1. 午前9時 2. 午前11時 3. 午後3時 4. 午後8時 B. ケーキと飲み物のセットを注文したいとき、どの時間に行くのが正しいですか。 1. 午前10時 2. 午後1時 3. 午後3時 4. 午後8時 C. アレルギーがあるお客様は、どのようにしたらよいですか。 1. メニューを見ない 2. 店員に伝える 3. 何も言わない 4. 他のお客さんに聞く D. 店員を呼びたいとき、どの表現が正しいですか。 1. 「こんにちは」と言う 2. 「すみません」と言う 3. 「ありがとう」と言う 4. 「ごちそうさま」と言う E. 次の会話の（　）に入る言葉として正しいものを選びなさい。 お客さん：「（　　　）、日替わりランチをください。」 店員：「かしこまりました。」 1. ありがとう 2. すみません 3. いらっしゃいませ 4. こんにちは --- #### 解答・解説 Aの正解：2（午前11時） **解説**：日替わりランチは「11:00-14:00」まで提供です。午前9時や午後3時、午後8時は提供時間外です。N3では時刻表現の理解がポイントです。 Bの正解：3（午後3時） **解説**：デザートセットは「14:00-17:00」に提供されます。午後3時はこの時間内です。午前や午後8時は時間外、午後1時もまだ提供前です。 Cの正解：2（店員に伝える） **解説**：「アレルギーがある方は、注文時に店員にお知らせください」と注意書きがあります。日本語で「伝える」「お知らせください」が読めるかがポイントです。 Dの正解：2（「すみません」と言う） **解説**：「ご注文はテーブルで『すみません』と店員を呼んでからお願いします」と書かれています。「すみません」はレストランで店員を呼ぶときの定番表現です。 Eの正解：2（すみません） **解説**：注文のときは「すみません、○○をください」と言うのが丁寧です。「ありがとう」は注文前には使いません。 --- #### 教師からの解説 今回の問題は、レストランの利用案内やメニュー表から必要な情報を探し、正しく理解・応用できるかを問う内容です。N3レベルの学習者にとって、時刻表現・注意書き・会話表現の読解が主なチャレンジポイントとなります。「すみません」の使い方や、アレルギー対応の案内も日本の実際のレストラン文化に即しており、実用的な日本語力を測る良い練習となります。
<p><strong>Q: 日替わりランチを食べたい場合、どの時間にレストランに行くのがよいですか。</strong></p>
<ul>
<li>午前9時</li>
<li><b>午前11時</b> <span style='color:green;'>(correct)</span></li>
<li>午後3時</li>
<li>午後8時</li>
</ul>
<p><em>Explanation: 日替わりランチは「11:00-14:00」まで提供です。午前9時や午後3時、午後8時は提供時間外です。N3では時刻表現の理解がポイントです。</em></p>
<p><strong>Q: ケーキと飲み物のセットを注文したいとき、どの時間に行くのが正しいですか。</strong></p>
<ul>
<li>午前10時</li>
<li>午後1時</li>
<li><b>午後3時</b> <span style='color:green;'>(correct)</span></li>
<li>午後8時</li>
</ul>
<p><em>Explanation: デザートセットは「14:00-17:00」に提供されます。午後3時はこの時間内です。午前や午後8時は時間外、午後1時もまだ提供前です。</em></p>
<p><strong>Q: アレルギーがあるお客様は、どのようにしたらよいですか。</strong></p>
<ul>
<li>メニューを見ない</li>
<li><b>店員に伝える</b> <span style='color:green;'>(correct)</span></li>
<li>何も言わない</li>
<li>他のお客さんに聞く</li>
</ul>
<p><em>Explanation: 「アレルギーがある方は、注文時に店員にお知らせください」と注意書きがあります。日本語で「伝える」「お知らせください」が読めるかがポイントです。</em></p>
<p><strong>Q: 店員を呼びたいとき、どの表現が正しいですか。</strong></p>
<ul>
<li>「こんにちは」と言う</li>
<li><b>「すみません」と言う</b> <span style='color:green;'>(correct)</span></li>
<li>「ありがとう」と言う</li>
<li>「ごちそうさま」と言う</li>
</ul>
<p><em>Explanation: 「ご注文はテーブルで『すみません』と店員を呼んでからお願いします」と書かれています。「すみません」はレストランで店員を呼ぶときの定番表現です。</em></p>
<p><strong>Q: 次の会話の（　）に入る言葉として正しいものを選びなさい。お客さん：「（　　　）、日替わりランチをください。」店員：「かしこまりました。」</strong></p>
<ul>
<li>ありがとう</li>
<li><b>すみません</b> <span style='color:green;'>(correct)</span></li>
<li>いらっしゃいませ</li>
<li>こんにちは</li>
</ul>
<p><em>Explanation: 注文のときは「すみません、○○をください」と言うのが丁寧です。「ありがとう」は注文前には使いません。</em></p>
<h1>第3部：聴解</h1>
<h2>話題理解 (topic_understanding)</h2>
<p>1問。会話を聞いて、話題が何かを選びなさい。</p>
<h3>道を尋ねる</h3>
<p><strong>Q: この会話の後、男の人はまず何をしますか。</strong></p>
<ul>
<li>右に曲がって公園に入る</li>
<li>バスに乗る</li>
<li><b>この道をまっすぐ歩く</b> <span style='color:green;'>(correct)</span></li>
<li>近くの人にもう一度道を聞く</li>
</ul>
<p><em>Explanation: この問題のポイントは、道案内の基本的な表現（「まっすぐ行って」「二つ目の信号を右に曲がる」など）を正しく聞き取れるかどうかです。選択肢の中には、すぐに右に曲がる（1）や公共交通機関を使う（2）、もう一度道を聞く（4）など、よくある混乱しやすいパターンが含まれています。正しい順序を聞き取り、最初の行動を選ぶことが求められます。N3レベルでは、道案内の会話を聞いて流れを理解できるかが重要なポイントです。</em></p>
</body></html>

In [29]:
# timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
# filename = f"./output/jlpt_simulator/JLPT_{timestamp}.html"

# with open(filename, "w", encoding="utf-8") as file:
#     file.write(html_output)